### Objective

**This is a simulation of customer transition year-to-year by state of engagement based on the following buckets**:

- **Loyal** *(At least 2 days of txns. in the last 365 days and at least 1 a year prior)*
- **Occasional** *(At least 1 day of txns. in the last 365 days)*
- **Dormant** *(No txn. in the last 365 days with at least 1 in the year prior)*
- **Lapsed** *(No txn. in the last 730 days)*


The expectation is to create a basis/status quo on which a Hidden Markov Model can be built in the near future (i.e. Simple Markov Chain) to better model customer transition from one state of engagement to another year-over-year.

[Creating the Function](#fxn) <br>
[Monte Carlo Simulation](#monte)<br>
[Credit for Code](#credit)<br>

In [1]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud import bigquery_storage

import re
import logging
import warnings
import time

from tqdm import tqdm

pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

%config Application.log_level = "ERROR"

warnings.filterwarnings(action='once')


def snakify(column_name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', column_name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

In [2]:
key_path = '/home/jupyter/d00_key.json'
credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

bq_client = bigquery.Client(credentials=credentials,
                            project=credentials.project_id)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

random_state = 777

QUERY = """SELECT
  customer_id,
  cust_state_now,
  cust_state_last_year,
  cust_state_2yrs_ago,
  cust_state_3yrs_ago
FROM
  `dw-bq-data-d00.SANDBOX_ANALYTICS.st_cust_state`"""
data = bq_client.query(QUERY).result().to_dataframe(
    bqstorage_client=bqstorageclient).sample(
        20000000, random_state=random_state).reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000000 entries, 0 to 19999999
Data columns (total 5 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   customer_id           int64 
 1   cust_state_now        object
 2   cust_state_last_year  object
 3   cust_state_2yrs_ago   object
 4   cust_state_3yrs_ago   object
dtypes: int64(1), object(4)
memory usage: 762.9+ MB


<a id='fxn'></a>

In [3]:
class CustomerXsition_v2(object):
    """
    Class to create a Markov transition matrix from a pandas DataFrame given the column
    that holds the observed states and make forecasts on future states based on empirical probabilities
    in the transition matrix.

    Monte Carlo simulations can be run using randomly generated sample data. 
    """

    def __init__(self, data, col1, col2):
        """
        Initialize the Markov Chain process using the underlying pandas DataFrame with
        the assumption that each possible state occurs in col1.

        Parameters
        ----------
        data : pandas DataFrame
            dataframe holding states in a column.
        col1 : str
            name of column holding current customer state.
        col2 : str
            name of column holding next customer state.
        """
        self.data = data
        self.col1 = col1
        self.col2 = col2
        self.states = sorted(list(self.data[self.col1].unique()))

        self.index_dict = {
            self.states[index]: index
            for index in range(len(self.states))
        }
        self.state_dict = {
            index: self.states[index]
            for index in range(len(self.states))
        }

    def make_matrix(self, transition_matrix=None):
        """
        Create the transition matrix from the underlying data or simply pass it if known.

        Parameters
        ----------
        transition_matrix: n-D array where n is the number of possible states
        """
        if transition_matrix:
            self.transition_matrix = transition_matrix
        else:
            self.transition_matrix = np.asarray([[
                len(self.data[(self.data[self.col1] == self.state_dict[i])
                              & (self.data[self.col2] == self.state_dict[j])])
                / len(self.data[self.data[self.col1] == self.state_dict[i]])
                for j in range(len(self.states))
            ] for i in range(len(self.states))])

        return self.transition_matrix

    def next_state(self, current_state):
        """
        Simulates the next state based on the current state and the probability of transitioning to
        other states.

        Parameters
        ----------
        current_state: str
            The customer's current state.
        """
        return np.random.choice(
            self.states,
            p=self.transition_matrix[self.index_dict[current_state], :])

    def generate_states(self, current_state, n=10):
        """
        Simulates the next n states based on the current state.

        Parameters
        ----------
        current_state: str
             The customer's current state.
        n: int
             The number of steps into the future to simulate.
        """
        future_states = []
        for i in range(n):
            next_state = self.next_state(current_state)
            future_states.append(next_state)
            current_state = next_state

        return future_states

    def monte_carlo_sim(self, samp_size=None, samp_n=0.05, n_sim=1000000):
        """
        Performs a Monte Carlo simulation using randomly generated samples of the data n_sim times.

        Parameters
        ----------
        samp_size : int
            size of desired sample
        samp_n : float
            percentage of data desired to create sample.
            Used by default when samp_size is not explicitly passed.
        n_sim : int
            no. of loops/simulations.
            Default is 1 million.
        """
        self.nested_dict = {
            self.states[index]:
            {self.states[index]: []
             for index in range(len(self.states))}
            for index in range(len(self.states))
        }

        for i in range(n_sim):
            if samp_size:
                self.sample = self.data.sample(samp_size)
            else:
                self.sample = self.data.sample(int(samp_n * len(self.data)))

            f_states = {
                state: [
                    self.next_state(state) for i in range(
                        len(self.sample[self.sample[column] == state]))
                ]
                for state in self.states
            }
            [[
                self.nested_dict[s][state].append(f_states[s].count(state))
                for state in self.states
            ] for s in self.states]

        for s in self.states:
            for state in self.states:
                print(
                    s, 'to', state + ':',
                    round(
                        np.mean(self.nested_dict[s][state]) /
                        len(self.sample[self.sample[column] == s]) * 100, 2),
                    '% chance')

In [4]:
print(CustomerXsition_v2.__doc__)


    Class to create a Markov transition matrix from a pandas DataFrame given the column
    that holds the observed states and make forecasts on future states based on empirical probabilities
    in the transition matrix.

    Monte Carlo simulations can be run using randomly generated sample data. 
    


In [5]:
print(CustomerXsition_v2.__init__.__doc__)


        Initialize the Markov Chain process using the underlying pandas DataFrame with
        the assumption that each possible state occurs in col1.

        Parameters
        ----------
        data : pandas DataFrame
            dataframe holding states in a column.
        col1 : str
            name of column holding current customer state.
        col2 : str
            name of column holding next customer state.
        


In [6]:
data.head()

,customer_id,cust_state_now,cust_state_last_year,cust_state_2yrs_ago,cust_state_3yrs_ago
0,1000180740772,Lapsed,Dormant,Occassional,Lapsed
1,1001245312650,Occassional,Lapsed,Lapsed,Lapsed
2,1000280723079,Occassional,Dormant,Occassional,Dormant
3,1000269938254,Lapsed,Dormant,Occassional,Dormant
4,1000905568518,Lapsed,Dormant,Occassional,Lapsed


In [7]:
cXsition_2019 = CustomerXsition_v2(data, 'cust_state_3yrs_ago', 'cust_state_2yrs_ago')
mx_2019 = cXsition_2019.make_matrix()

In [8]:
mx_2019

array([[0.        , 0.72211647, 0.        , 0.27788353],
       [0.        , 0.58711359, 0.        , 0.41288641],
       [0.2052242 , 0.        , 0.61684554, 0.17793025],
       [0.55706811, 0.        , 0.17427535, 0.26865655]])

In [9]:
cXsition_2019.states

['Dormant', 'Lapsed', 'Loyal', 'Occassional']

In [10]:
for state in cXsition_2019.states:
    print('The current state is:', state, '\nA simulated next state is:',
          cXsition_2019.next_state(state))
    print('\n')

The current state is: Dormant 
A simulated next state is: Lapsed


The current state is: Lapsed 
A simulated next state is: Occassional


The current state is: Loyal 
A simulated next state is: Loyal


The current state is: Occassional 
A simulated next state is: Occassional




In [11]:
cXsition_2020 = CustomerXsition_v2(data, 'cust_state_2yrs_ago', 'cust_state_last_year')
mx_2020 = cXsition_2020.make_matrix()

In [12]:
mx_2020

array([[0.        , 0.75659257, 0.        , 0.24340743],
       [0.        , 0.63214867, 0.        , 0.36785133],
       [0.2435606 , 0.        , 0.55291281, 0.20352659],
       [0.61908867, 0.        , 0.13332733, 0.247584  ]])

In [13]:
cXsition_2020.states

['Dormant', 'Lapsed', 'Loyal', 'Occassional']

In [14]:
cXsition_2021 = CustomerXsition_v2(data, 'cust_state_last_year', 'cust_state_now')
mx_2021 = cXsition_2021.make_matrix()

In [15]:
mx_2021

array([[0.        , 0.77166951, 0.        , 0.22833049],
       [0.        , 0.62057365, 0.        , 0.37942635],
       [0.26450016, 0.        , 0.49960939, 0.23589045],
       [0.61787869, 0.        , 0.11651006, 0.26561124]])

In [16]:
cXsition_2021.states

['Dormant', 'Lapsed', 'Loyal', 'Occassional']

In [17]:
# simulating a customer's state in 2019...

data['pred_state_2019'] = data['state_2018'].apply(cXsition.next_state)

In [18]:
data.head()

,address_id,freq_2016,freq_2017,freq_2018,freq_2019,state_2017,state_2018,state_2019,pred_state_2019
0,3000049708318,nan,nan,nan,nan,lapsed,lapsed,lapsed,lapsed
1,3000056367699,nan,nan,nan,nan,lapsed,lapsed,lapsed,occassional
2,3000068725008,nan,nan,nan,nan,lapsed,lapsed,lapsed,lapsed
3,3000396727694,1.000,nan,nan,1.000,dormant,lapsed,occassional,lapsed
4,3000148387504,nan,nan,nan,nan,lapsed,lapsed,lapsed,lapsed


In [19]:
data['state_2019'].value_counts()

lapsed         6511953
occassional    1270240
active         1268657
dormant         949150
Name: state_2019, dtype: int64

In [20]:
data['pred_state_2019'].value_counts()

lapsed         6489567
active         1340758
occassional    1288129
dormant         881546
Name: pred_state_2019, dtype: int64

In [21]:
print(states)
confusion_matrix(data['state_2019'].values,
                 data['pred_state_2019'].values,
                 labels=states)

['active', 'occassional', 'dormant', 'lapsed']


array([[ 810089,  153659,  262112,   42797],
       [ 168473,  238869,  208928,  653970],
       [ 314535,  224109,  410506,       0],
       [  47661,  671492,       0, 5792800]])

In [22]:
multilabel_confusion_matrix(data['state_2019'],
                            data['pred_state_2019'],
                            labels=states)

array([[[8200674,  530669],
        [ 458568,  810089]],

       [[7680500, 1049260],
        [1031371,  238869]],

       [[8579810,  471040],
        [ 538644,  410506]],

       [[2791280,  696767],
        [ 719153, 5792800]]])

In [23]:
print(
    classification_report(data['state_2019'].values,
                          data['pred_state_2019'].values,
                          labels=states,
                          target_names=states))

              precision    recall  f1-score   support

      active       0.60      0.64      0.62   1268657
 occassional       0.19      0.19      0.19   1270240
     dormant       0.47      0.43      0.45    949150
      lapsed       0.89      0.89      0.89   6511953

    accuracy                           0.73  10000000
   macro avg       0.54      0.54      0.54  10000000
weighted avg       0.73      0.73      0.73  10000000



<a id='monte'></a>

### Monte Carlo Simulation

In [24]:
def monte_sim(data, column, samp_size=None, samp_n=0.1, n_sim=1000):
    states = list(data[column].unique())

    nested_dict = {
        states[index]: {states[index]: []
                        for index in range(len(states))}
        for index in range(len(states))
    }
    
    print('Starting simulation...')
    
    for i in tqdm(list(range(n_sim))):
        if samp_size:
            sample = data.sample(samp_size)
        else:
            sample = data.sample(int(samp_n * len(data)))

        f_states = {
            state: [
                cXsition.next_state(state)
                for j in range(len(sample[sample[column] == state]))
            ]
            for state in states
        }
        [[
            nested_dict[s][state].append(f_states[s].count(state))
            for state in states
        ] for s in states]
        
    print('Results of simulation stored.')
    print('\n')

    for s in states:
        for state in states:
            print(
                s, 'to', state + ':',
                round(
                    np.mean(nested_dict[s][state]) /
                    len(sample[sample[column] == s]) * 100, 4), '% chance')
            
    print('\n')        
    print('Simulation complete!')

In [25]:
monte_sim(data=data, column='state_2017', samp_n=0.05, n_sim=100)

  0%|          | 0/100 [00:00<?, ?it/s]

Starting simulation...


100%|██████████| 100/100 [28:51<00:00, 17.31s/it]


Results of simulation stored.


lapsed to lapsed: 91.2014 % chance
lapsed to dormant: 0.0 % chance
lapsed to occassional: 8.2535 % chance
lapsed to active: 0.4345 % chance
dormant to lapsed: 64.4818 % chance
dormant to dormant: 0.0 % chance
dormant to occassional: 31.7455 % chance
dormant to active: 3.8894 % chance
occassional to lapsed: 0.0 % chance
occassional to dormant: 53.1169 % chance
occassional to occassional: 29.1351 % chance
occassional to active: 17.9672 % chance
active to lapsed: 0.0 % chance
active to dormant: 15.1568 % chance
active to occassional: 8.2282 % chance
active to active: 76.871 % chance


Simulation complete!


In [ ]:
monte_sim(data=data, column='state_2017', samp_n=0.01, n_sim=10000)

  0%|          | 0/10000 [00:00<?, ?it/s]

Starting simulation...


  3%|▎         | 275/10000 [18:30<10:54:23,  4.04s/it]

<a id='credit'></a>

Credit for basis of code:<br>
**Alessandro Molina on Medium (Markov Chains with Python)** <br>
https://medium.com/@__amol__/markov-chains-with-python-1109663f3678